# Predspracovanie dát

In [18]:
import sys
import os

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))

from pyspark.ml.feature import StringIndexer , OneHotEncoder, Bucketizer, VectorAssembler, StandardScaler
from pyspark.sql.functions import col, log2, sum, count, when
from integration.integration import integration
from pyspark.sql.functions import median
from pyspark.sql import SparkSession
from collections import Counter

In [19]:
spark = SparkSession.builder.appName("preprocessing").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

In [20]:
df_train, df_test = integration()
df_train = df_train.drop("Accident_Index", "Date")
df_test = df_test.drop("Accident_Index","Date")

#### Zistenie, ktoré stĺpce sú kvantitatívne a kategorické

In [6]:
types = df_train.dtypes
print(types)

[('Location_Easting_OSGR', 'int'), ('Location_Northing_OSGR', 'int'), ('Longitude', 'double'), ('Latitude', 'double'), ('Police_Force', 'int'), ('Accident_Severity', 'int'), ('Number_of_Vehicles', 'int'), ('Number_of_Casualties', 'int'), ('Day_of_Week', 'int'), ('Time', 'timestamp'), ('Local_Authority_(District)', 'int'), ('Local_Authority_(Highway)', 'string'), ('1st_Road_Class', 'int'), ('1st_Road_Number', 'int'), ('Road_Type', 'int'), ('Speed_limit', 'int'), ('Junction_Detail', 'int'), ('Junction_Control', 'int'), ('2nd_Road_Class', 'int'), ('2nd_Road_Number', 'int'), ('Pedestrian_Crossing-Human_Control', 'int'), ('Pedestrian_Crossing-Physical_Facilities', 'int'), ('Light_Conditions', 'int'), ('Weather_Conditions', 'int'), ('Road_Surface_Conditions', 'int'), ('Special_Conditions_at_Site', 'int'), ('Carriageway_Hazards', 'int'), ('Urban_or_Rural_Area', 'int'), ('Did_Police_Officer_Attend_Scene_of_Accident', 'int'), ('LSOA_of_Accident_Location', 'string'), ('Vehicle_Reference', 'int')

In [7]:
counts = Counter()

for dtype in types:
    counts[dtype[1]] += 1

print(counts)

Counter({'int': 59, 'double': 2, 'string': 2, 'timestamp': 1})


#### Vypočítanie základných štatistík pre kvantitatívne stĺpce

In [8]:
def get_int(df):
    int_columns = []

    for column, data_type in df.dtypes:
        if data_type == "int":
            int_columns.append(column)
    return int_columns

In [9]:
def show_stats(df, columns, batch_size=4):
    for i in range(0, len(columns), batch_size):
        df.describe(columns[i:i+batch_size]).show()

show_stats(df_train, get_int(df_train))

+-------+---------------------+----------------------+------------------+------------------+
|summary|Location_Easting_OSGR|Location_Northing_OSGR|      Police_Force| Accident_Severity|
+-------+---------------------+----------------------+------------------+------------------+
|  count|               300276|                300276|            300293|            300293|
|   mean|      438255.11356885|    300105.18741757586| 30.78688480916971|2.8231294102759636|
| stddev|    94748.83907367043|    157397.67714086492|24.743146328252084|0.4288013609691326|
|    min|                65860|                 10560|                 1|                 1|
|    max|               655370|               1191500|                98|                 3|
+-------+---------------------+----------------------+------------------+------------------+

+-------+------------------+--------------------+------------------+--------------------------+
|summary|Number_of_Vehicles|Number_of_Casualties|       Day_of_Wee

#### Zobrazenie histogramu pre kategorické atribúty

In [10]:
df_train.groupby("Local_Authority_(Highway)").count().orderBy("count", ascending=False).show()

+-------------------------+-----+
|Local_Authority_(Highway)|count|
+-------------------------+-----+
|                E10000016| 9067|
|                E10000030| 8536|
|                E10000017| 7218|
|                E10000012| 6495|
|                E10000015| 6219|
|                E08000025| 5805|
|                E10000014| 5748|
|                E10000028| 5051|
|                E10000019| 4289|
|                E08000035| 4230|
|                E10000024| 4218|
|                E10000007| 4170|
|                E10000020| 3831|
|                E10000032| 3822|
|                E10000023| 3772|
|                E10000008| 3671|
|                E10000029| 3628|
|                E10000003| 3550|
|                E10000025| 3258|
|                E08000032| 3116|
+-------------------------+-----+
only showing top 20 rows



In [11]:
df_train.groupby("LSOA_of_Accident_Location").count().orderBy("count", ascending=False).show()

+-------------------------+-----+
|LSOA_of_Accident_Location|count|
+-------------------------+-----+
|                     NULL|19875|
|                E01024597|  335|
|                E01000004|  308|
|                E01018648|  252|
|                E01023722|  251|
|                E01011365|  244|
|                E01023584|  211|
|                E01030458|  190|
|                E01023725|  170|
|                E01004736|  164|
|                E01005131|  160|
|                E01018465|  154|
|                E01016481|  153|
|                E01002444|  152|
|                E01029317|  148|
|                E01023732|  135|
|                E01018337|  130|
|                E01006650|  128|
|                E01017620|  128|
|                E01008440|  127|
+-------------------------+-----+
only showing top 20 rows



## Chýbajúce hodnoty

In [4]:
df_train.select([count(when(col(c).isNull(), c)).alias(c) for c in df_train.columns]).show()

+---------------------+----------------------+---------+--------+------------+-----------------+------------------+--------------------+-----------+----+--------------------------+-------------------------+--------------+---------------+---------+-----------+---------------+----------------+--------------+---------------+---------------------------------+---------------------------------------+----------------+------------------+-----------------------+--------------------------+-------------------+-------------------+-------------------------------------------+-------------------------+-----------------+------------------+--------------+---------------+---------------+--------------------+-----------------+-------------------+-------------------+-------------+----------------------+----------------------------------+-------------+-----------------------+------------+-----------------------+-----------------+--------------------------------+-----------------+------------------------+--

## Odstranenie atribútov

In [21]:
df_train = df_train.drop("Accident_Index", "Date", "Time", "Longitude", "Latitude", "LSOA_of_Accident_Location")
df_test = df_test.drop("Accident_Index","Date", "Time", "Longitude", "Latitude", "LSOA_of_Accident_Location")

## Nahradenie chýbajúcich hodnôt mediánom

In [22]:
def fillna_median(df, include=None):
    if include is None:
        include = set()
    medians = df.agg(*(median(x).alias(x) for x in df.columns if x in include))
    return df.fillna(medians.first().asDict())


df_train = fillna_median(df_train, ['Location_Easting_OSGR', 'Location_Northing_OSGR'])
df_test  = fillna_median(df_test, ['Location_Easting_OSGR', 'Location_Northing_OSGR'])

In [15]:
df_train.select([count(when(col(c).isNull(), c)).alias(c) for c in df_train.columns]).show()

+---------------------+----------------------+------------+-----------------+------------------+--------------------+-----------+--------------------------+-------------------------+--------------+---------------+---------+-----------+---------------+----------------+--------------+---------------+---------------------------------+---------------------------------------+----------------+------------------+-----------------------+--------------------------+-------------------+-------------------+-------------------------------------------+-----------------+------------------+--------------+---------------+---------------+--------------------+-----------------+-------------------+-------------------+-------------+----------------------+----------------------------------+-------------+-----------------------+------------+-----------------------+-----------------+--------------------------------+-----------------+------------------------+-------------------------+--------------------------

## Nominálne na kvantitatívne pomocou One-hot encoding

In [23]:
categorical_attributes = [x for x, y in df_train.dtypes if y == 'string']
index_output_cols = [x + "_SI" for x in categorical_attributes]
ohe_output_cols  = [x + "_OHE" for x in categorical_attributes]

indexer = StringIndexer(inputCols=categorical_attributes , outputCols=index_output_cols, handleInvalid="skip")
df_train = indexer.fit(df_train).transform(df_train)

ohe = OneHotEncoder(inputCols=index_output_cols ,outputCols =ohe_output_cols)
df_train = ohe.fit(df_train).transform(df_train)

In [8]:
df_train = df_train.drop(*index_output_cols)
df_train = df_train.drop(*categorical_attributes)

df_train.show(5)

+---------------------+----------------------+------------+-----------------+------------------+--------------------+-----------+--------------------------+--------------+---------------+---------+-----------+---------------+----------------+--------------+---------------+---------------------------------+---------------------------------------+----------------+------------------+-----------------------+--------------------------+-------------------+-------------------+-------------------------------------------+-----------------+------------------+--------------+---------------+---------------+--------------------+-----------------+-------------------+-------------------+-------------+----------------------+----------------------------------+-------------+-----------------------+------------+-----------------------+-----------------+--------------------------------+-----------------+------------------------+-------------------------+---------------------------+------------------------

## Náhrada numerických atribútov nominálnymi

In [24]:
driver_age_splits = [-float("inf"), 18, 25, 40, 60, float("inf")]
driver_age_bucketizer = Bucketizer(splits=driver_age_splits, inputCol="Age_of_Driver", outputCol="Age_of_Driver_Cat", handleInvalid="skip")

vehicle_age_splits = [-float("inf"), 0, 2, 6, 11, float("inf")]
vehicle_age_bucketizer = Bucketizer(splits=vehicle_age_splits, inputCol="Age_of_Vehicle", outputCol="Age_of_Vehicle_Cat", handleInvalid="skip")

engine_capacity_splits = [-float("inf"), -1, 0, 1000, 2000, 3000, 5000, 8000, float("inf")]
engine_bucketizer = Bucketizer(splits=engine_capacity_splits, inputCol="Engine_Capacity_(CC)", outputCol="Engine_Size_Cat", handleInvalid="skip")

vehicle_type_splits = [-float("inf"), 0, 1.5, 2.5, 3.5, 4.5, 5.5, 7, 8.5, 9.5, 10.5, 11.5, 16.5, 17.5, 18.5, 19.5, 20.5, 21.5, 90.5, float("inf")]
vehicle_type_bucketizer = Bucketizer(splits=vehicle_type_splits, inputCol="Vehicle_Type", outputCol="Vehicle_Type_Cat", handleInvalid="keep")

for col_name in ["Age_of_Driver_Cat", "Age_of_Vehicle_Cat", "Engine_Size_Cat", "Vehicle_Type_Cat"]:
    if col_name in df_train.columns:
        df_train = df_train.drop(col_name)
    if col_name in df_test.columns:
        df_test = df_test.drop(col_name)

df_train = driver_age_bucketizer.transform(df_train)
df_train = vehicle_age_bucketizer.transform(df_train)
df_train = engine_bucketizer.transform(df_train)
df_train = vehicle_type_bucketizer.transform(df_train)

df_test = driver_age_bucketizer.transform(df_test)
df_test = vehicle_age_bucketizer.transform(df_test)
df_test = engine_bucketizer.transform(df_test)
df_test = vehicle_type_bucketizer.transform(df_test)

print("DataFrame schema after binning:")
df_train.printSchema()
print("Example data after binning:")
df_train.select(
    "Age_of_Driver", "Age_of_Driver_Cat", 
    "Age_of_Vehicle", "Age_of_Vehicle_Cat",
    "Engine_Capacity_(CC)", "Engine_Size_Cat",
    "Vehicle_Type", "Vehicle_Type_Cat"
).show(5)

df_train = df_train.drop("Age_of_Driver", "Age_of_Vehicle", "Engine_Capacity_(CC)", "Vehicle_Type")
df_test = df_test.drop("Age_of_Driver", "Age_of_Vehicle", "Engine_Capacity_(CC)", "Vehicle_Type")

DataFrame schema after binning:
root
 |-- Location_Easting_OSGR: integer (nullable = true)
 |-- Location_Northing_OSGR: integer (nullable = true)
 |-- Police_Force: integer (nullable = true)
 |-- Accident_Severity: integer (nullable = true)
 |-- Number_of_Vehicles: integer (nullable = true)
 |-- Number_of_Casualties: integer (nullable = true)
 |-- Day_of_Week: integer (nullable = true)
 |-- Local_Authority_(District): integer (nullable = true)
 |-- Local_Authority_(Highway): string (nullable = true)
 |-- 1st_Road_Class: integer (nullable = true)
 |-- 1st_Road_Number: integer (nullable = true)
 |-- Road_Type: integer (nullable = true)
 |-- Speed_limit: integer (nullable = true)
 |-- Junction_Detail: integer (nullable = true)
 |-- Junction_Control: integer (nullable = true)
 |-- 2nd_Road_Class: integer (nullable = true)
 |-- 2nd_Road_Number: integer (nullable = true)
 |-- Pedestrian_Crossing-Human_Control: integer (nullable = true)
 |-- Pedestrian_Crossing-Physical_Facilities: integer (n

## Informačný zisk

In [ ]:
from pyspark.sql import functions as F
import matplotlib.pyplot as plt

def calculate_entropy(df, group_col):
    """Calculates the entropy for a given column."""
    total_count = df.count()
    entropy = df.groupBy(group_col) \
                .agg((F.count("*") / total_count).alias("prob")) \
                .select(F.sum(when(col("prob") > 0, -col("prob") * log2(col("prob")))).alias("entropy")) \
                .first()["entropy"]
    if entropy is None:
        return 0.0
    return entropy

def calculate_information_gain(df, attribute_col, target_col):
    """Calculates the Information Gain of an attribute column with respect to a target column."""
    total_count = df.count()

    total_entropy = calculate_entropy(df, target_col)

    conditional_entropy_df = df.groupBy(attribute_col, target_col) \
                               .agg(F.count("*").alias("attr_target_count")) \
                               .join(df.groupBy(attribute_col).agg(F.count("*").alias("attr_count")), on=attribute_col) \
                               .withColumn("cond_prob", col("attr_target_count") / col("attr_count")) \
                               .select(attribute_col, "attr_count",
                                       when(col("cond_prob") > 0, -col("cond_prob") * log2(col("cond_prob"))).alias("term")) \
                               .groupBy(attribute_col, "attr_count") \
                               .agg(F.sum("term").alias("entropy_attr_val")) \
                               .withColumn("attr_prob", col("attr_count") / total_count) \
                               .select(F.sum(col("attr_prob") * col("entropy_attr_val")).alias("conditional_entropy"))

    conditional_entropy = conditional_entropy_df.first()["conditional_entropy"]
    if conditional_entropy is None:
        conditional_entropy = 0.0

    information_gain = total_entropy - conditional_entropy
    return information_gain

target_column = "Accident_Severity"

cols_for_ig = [col for col in df_train.columns if col != target_column]

print(f"Calculating Information Gain with respect to '{target_column}':")

ig_results = {}


for attr_col in cols_for_ig:
    if attr_col in df_train.columns:
        ig = calculate_information_gain(df_train, attr_col, target_column)
        ig_results[attr_col] = ig
        print(f"  IG({attr_col}): {ig:.4f}")
    else:
        print(f"  Column '{attr_col}' not found in df_train. Skipping.")


# Plotting the top 10 attributes with the highest Information Gain
sorted_ig = sorted(ig_results.items(), key=lambda x: x[1], reverse=True)[:10]
attributes, gains = zip(*sorted_ig)

plt.figure(figsize=(10, 6))
plt.barh(attributes, gains)
plt.xlabel("Information Gain")
plt.title("Top 10 atribútov podľa Information Gain")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## Normalizácia pomocou StandardScaler - Vypočítanie štatistík pre numerické atribúty

In [25]:
numerical_cols_to_scale = [
    'Location_Easting_OSGR',
    'Location_Northing_OSGR',
    'Police_Force',
    'Accident_Severity',
    'Number_of_Vehicles',
    'Number_of_Casualties',
    'Day_of_Week',
    'Local_Authority_(District)',
    '1st_Road_Class',
    '1st_Road_Number',
    'Road_Type',
    'Speed_limit',
    'Junction_Detail',
    'Junction_Control',
    '2nd_Road_Class',
    '2nd_Road_Number',
    'Pedestrian_Crossing-Human_Control',
    'Pedestrian_Crossing-Physical_Facilities',
    'Light_Conditions',
    'Weather_Conditions',
    'Road_Surface_Conditions',
    'Special_Conditions_at_Site',
    'Carriageway_Hazards',
    'Urban_or_Rural_Area',
    'Did_Police_Officer_Attend_Scene_of_Accident',
    'Vehicle_Reference',
    'Casualty_Reference',
    'Casualty_Class',
    'Sex_of_Casualty',
    'Age_of_Casualty',
    'Age_Band_of_Casualty',
    'Casualty_Severity',
    'Pedestrian_Location',
    'Pedestrian_Movement',
    'Car_Passenger',
    'Bus_or_Coach_Passenger',
    'Pedestrian_Road_Maintenance_Worker',
    'Casualty_Type',
    'Casualty_Home_Area_Type',
    'Towing_and_Articulation',
    'Vehicle_Manoeuvre',
    'Vehicle_Location-Restricted_Lane',
    'Junction_Location',
    'Skidding_and_Overturning',
    'Hit_Object_in_Carriageway',
    'Vehicle_Leaving_Carriageway',
    'Hit_Object_off_Carriageway',
    '1st_Point_of_Impact',
    'Was_Vehicle_Left_Hand_Drive?',
    'Journey_Purpose_of_Driver',
    'Sex_of_Driver',
    'Age_Band_of_Driver',
    'Propulsion_Code',
    'Driver_IMD_Decile',
    'Driver_Home_Area_Type'
]

# Filter out columns that might not be present in the DataFrame at this stage
existing_numerical_cols = [col_name for col_name, dtype in df_train.dtypes 
                          if col_name in numerical_cols_to_scale and dtype in ['int', 'double']]

if not existing_numerical_cols:
    print("No numerical columns found for scaling.")
else:
    print(f"Numerical columns selected for scaling: {existing_numerical_cols}")

    assembler_numerical = VectorAssembler(
        inputCols=existing_numerical_cols,
        outputCol="numerical_features_vec",
        handleInvalid="skip" 
    )
    df_train = assembler_numerical.transform(df_train)
    df_test = assembler_numerical.transform(df_test)

    scaler = StandardScaler(
        inputCol="numerical_features_vec",
        outputCol="scaled_numerical_features",
        withStd=True, 
        withMean=True 
    )

    scaler_model = scaler.fit(df_train)

    df_train = scaler_model.transform(df_train)
    df_test = scaler_model.transform(df_test)

    print("Numerical features scaled using StandardScaler.")
    df_train.select("numerical_features_vec", "scaled_numerical_features").show(5, truncate=False)

Numerical columns selected for scaling: ['Location_Easting_OSGR', 'Location_Northing_OSGR', 'Police_Force', 'Accident_Severity', 'Number_of_Vehicles', 'Number_of_Casualties', 'Day_of_Week', 'Local_Authority_(District)', '1st_Road_Class', '1st_Road_Number', 'Road_Type', 'Speed_limit', 'Junction_Detail', 'Junction_Control', '2nd_Road_Class', '2nd_Road_Number', 'Pedestrian_Crossing-Human_Control', 'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions', 'Weather_Conditions', 'Road_Surface_Conditions', 'Special_Conditions_at_Site', 'Carriageway_Hazards', 'Urban_or_Rural_Area', 'Did_Police_Officer_Attend_Scene_of_Accident', 'Vehicle_Reference', 'Casualty_Reference', 'Casualty_Class', 'Sex_of_Casualty', 'Age_of_Casualty', 'Age_Band_of_Casualty', 'Casualty_Severity', 'Pedestrian_Location', 'Pedestrian_Movement', 'Car_Passenger', 'Bus_or_Coach_Passenger', 'Pedestrian_Road_Maintenance_Worker', 'Casualty_Type', 'Casualty_Home_Area_Type', 'Towing_and_Articulation', 'Vehicle_Manoeuvre', 'Veh

## Odstránenie irelevantných atribútov

In [26]:
THRESHOLD = 0.009

#low_ig_cols = [col for col, ig in ig_results.items() if ig < THRESHOLD]

low_ig_cols = [
    "Day_of_Week",
    "1st_Road_Class",
    "Road_Type",
    "Pedestrian_Crossing-Human_Control",
    "Pedestrian_Crossing-Physical_Facilities",
    "Light_Conditions",
    "Weather_Conditions",
    "Road_Surface_Conditions",
    "Special_Conditions_at_Site",
    "Carriageway_Hazards",
    "Casualty_Class",
    "Sex_of_Casualty",
    "Age_of_Casualty",
    "Age_Band_of_Casualty",
    "Pedestrian_Location",
    "Pedestrian_Movement",
    "Car_Passenger",
    "Bus_or_Coach_Passenger",
    "Pedestrian_Road_Maintenance_Worker",
    "Casualty_Home_Area_Type",
    "Towing_and_Articulation",
    "Vehicle_Location-Restricted_Lane",
    "Skidding_and_Overturning",
    "Hit_Object_in_Carriageway",
    "Hit_Object_off_Carriageway",
    "Was_Vehicle_Left_Hand_Drive?",
    "Journey_Purpose_of_Driver",
    "Sex_of_Driver",
    "Age_of_Driver",
    "Age_Band_of_Driver",
    "Propulsion_Code",
    "Age_of_Vehicle",
    "Driver_IMD_Decile",
    "Driver_Home_Area_Type",
    "Age_of_Driver_Cat",
    "Age_of_Vehicle_Cat",
    "Engine_Size_Cat"
]

df_train = df_train.drop(*low_ig_cols)
df_test = df_test.drop(*low_ig_cols)

#print("Columns with IG < 0.009 were removed:")
#for col in low_ig_cols:
#    print(f" - {col} (IG: {ig_results[col]:.4f})")


In [27]:
print("Remaining columns after filtering (IG ≥ 0.009):")
for col_name in df_train.columns:
    print(f" - {col_name}")

Remaining columns after filtering (IG ≥ 0.009):
 - Location_Easting_OSGR
 - Location_Northing_OSGR
 - Police_Force
 - Accident_Severity
 - Number_of_Vehicles
 - Number_of_Casualties
 - Local_Authority_(District)
 - Local_Authority_(Highway)
 - 1st_Road_Number
 - Speed_limit
 - Junction_Detail
 - Junction_Control
 - 2nd_Road_Class
 - 2nd_Road_Number
 - Urban_or_Rural_Area
 - Did_Police_Officer_Attend_Scene_of_Accident
 - Vehicle_Reference
 - Casualty_Reference
 - Casualty_Severity
 - Casualty_Type
 - Vehicle_Manoeuvre
 - Junction_Location
 - Vehicle_Leaving_Carriageway
 - 1st_Point_of_Impact
 - Local_Authority_(Highway)_SI
 - Local_Authority_(Highway)_OHE
 - Vehicle_Type_Cat
 - numerical_features_vec
 - scaled_numerical_features


# PASS DO MODELU

In [31]:
# Get all remaining column names after filtering by IG threshold
remaining_cols = df_train.columns

# Remove any special columns like target variable (Accident_Severity) and already vectorized features
exclude_cols = ["Accident_Severity", "numerical_features_vec", "scaled_numerical_features", 
                "Local_Authority_(Highway)", "Local_Authority_(Highway)_SI",
                "Local_Authority_(Highway)_OHE"]
feature_cols = [col for col in remaining_cols if col not in exclude_cols]

print("Creating feature vector with the following columns:")
for col in feature_cols:
    print(f" - {col}")

# Create the final feature vector for modeling
assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features",
    handleInvalid="skip"
)

# Transform both train and test datasets
df_train_model = assembler.transform(df_train)
df_test_model = assembler.transform(df_test)

# Show the resulting dataframe with the features vector
print("\nExample of the final dataset:")
df_train_model.select("features", "Accident_Severity").show(5, truncate=False)

# Save the preprocessed data for modeling
print("\nPreprocessed datasets created successfully!")
print(f"Training set size: {df_train_model.count()} rows")
print(f"Test set size: {df_test_model.count()} rows")

Creating feature vector with the following columns:
 - Location_Easting_OSGR
 - Location_Northing_OSGR
 - Police_Force
 - Number_of_Vehicles
 - Number_of_Casualties
 - Local_Authority_(District)
 - 1st_Road_Number
 - Speed_limit
 - Junction_Detail
 - Junction_Control
 - 2nd_Road_Class
 - 2nd_Road_Number
 - Urban_or_Rural_Area
 - Did_Police_Officer_Attend_Scene_of_Accident
 - Vehicle_Reference
 - Casualty_Reference
 - Casualty_Severity
 - Casualty_Type
 - Vehicle_Manoeuvre
 - Junction_Location
 - Vehicle_Leaving_Carriageway
 - 1st_Point_of_Impact
 - Vehicle_Type_Cat

Example of the final dataset:
+---------------------------------------------------------------------------------------------------------------+-----------------+
|features                                                                                                       |Accident_Severity|
+---------------------------------------------------------------------------------------------------------------+-----------------+
|

In [34]:
processed_dir = os.path.join("..", "processed_data")
os.makedirs(processed_dir, exist_ok=True)

train_path = os.path.join(processed_dir, "df_train_model.csv")
test_path = os.path.join(processed_dir, "df_test_model.csv")

df_train_model.write.mode("overwrite").csv(train_path)
df_test_model.write.mode("overwrite").csv(test_path)

print(f"Saved processed dataframes to {processed_dir}")

AnalysisException: [UNSUPPORTED_DATA_TYPE_FOR_DATASOURCE] The CSV datasource doesn't support the column `Local_Authority_(Highway)_OHE` of the type "STRUCT<type: TINYINT, size: INT, indices: ARRAY<INT>, values: ARRAY<DOUBLE>>".

# TODO:
- [x] Informačný zisk - F
- [x] Nahradiť chýbajúce hodnoty - M
- [x] Kvantitívne atribúty na nominálne (neviem prečo) - F
- [x] Normalizácia - StandardScaler - F
- [x] One-hot encoding - M
- [x] Odstrániť irelevantné atribúty - J